# Initialization

In [197]:
from src.SIV_library.lib import SIV, OpticalFlow
from src.SIV_library.plots import plot_flow
from src.SIV_library.advanced import ctf_optical, match_refine, Warp
from torchPIV import OfflinePIV

import os
import sys
import torch
import matplotlib.pyplot as plt

import pandas as pd
from datetime import datetime

plt.rcParams["animation.html"] = "jshtml"
%matplotlib notebook
# %matplotlib inline

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('device:', device)

if device == "cuda":
    torch.cuda.empty_cache()
    
folder = r"../Test Data/THE REAL DEAL/Processed_All"

device: cuda


In [149]:
plt.close('all')

# TorchPIV

In [42]:
piv_gen = OfflinePIV(
    folder=folder, # Path to experiment
    device=torch.cuda.get_device_name(), # Device name
    file_fmt="jpg",
    wind_size=64,
    overlap=32,
    dt=12, # Time between frames, mcs
    scale = 0.02, # mm/pix
    multipass=2,
    multipass_mode="DWS", # CWS or DWS
    multipass_scale=2.0, # Window downscale on each pass
    folder_mode="sequential" # Pairs or sequential frames 
)

for out in piv_gen():
    x, y, vx, vy = out
    x, y = x/piv_gen._scale, y/piv_gen._scale

Load time 0.034 sec Iteration finished in 0.103 sec Batch finished in 0.240 sec


In [49]:
plt.close('all')

fig, ax = plt.subplots(1, 1)
 
frame = piv_gen._dataset[0][0]
ax.imshow(frame, cmap='gray')
ax.quiver(x, y, vx, -vy, color='red', scale=0.05, scale_units='xy', angles='xy')

ax.set_title(f'TorchPIV')
ax.set_axis_off()

fig.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

# Matching

In [165]:
mode = 1

siv_gen = SIV(
    folder=folder, 
    device=device, 
    window_size=128, 
    overlap=64, 
    search_area=(50, 0, 0, 0), 
    mode=mode, 
    num_passes=2, 
    scale_factor=0.5
)

In [166]:
idx = 0
for res in siv_gen():
    x, y, u, v = res
    
    if idx == 0:
        x0 = torch.zeros((len(siv_gen), *x.shape))
        y0, u0, v0 = torch.zeros_like(x0), torch.zeros_like(x0), torch.zeros_like(x0)
    
    x0[idx], y0[idx], u0[idx], v0[idx] = x, y, u, v
    idx += 1

SAD: 100%|██████████| 1619/1619 [03:43<00:00,  7.25it/s]


In [155]:
plt.close('all')

idx = 0
fig, ax = plt.subplots(1, 1)

x0, y0, u0, v0 = x0.cpu(), y0.cpu(), u0.cpu(), v0.cpu()
 
frame = t.dataset[idx][0].cpu().numpy()
ax.imshow(frame, cmap='gray')
ax.quiver(x0[idx].numpy(), y0[idx].numpy(), u0[idx].numpy(), -v0[idx].numpy(), color='red', scale=1., scale_units='xy', angles='xy')

ax.set_title(f'Template Matching - {"SAD" if mode == 1 else "Correlation"}')
ax.set_axis_off()

fig.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

Interpolate velocity field:

In [158]:
w = Warp(x0.cpu(), y0.cpu(), u0.cpu(), v0.cpu())
w.interpolate_field(siv_gen.dataset.img_shape)
x1, y1, u1, v1 = w.x, w.y, w.u, w.v

In [166]:
# plt.close('all')
_ = plot_flow(u1 * factor, v1 * factor, grid_spacing=32, fn=None)

<IPython.core.display.Javascript object>

# Optical flow

In [135]:
opti_gen = OpticalFlow(
    folder=folder, 
    device=device, 
    alpha=100., 
    num_iter=10, 
    eps=0.
)

In [137]:
idx = 0
for res in opti_gen():
    x, y, u, v = res
  
    if idx == 0:
        x2 = torch.zeros((len(opti_gen), *x.shape))
        y2, u2, v2 = torch.zeros_like(x2), torch.zeros_like(x2), torch.zeros_like(x2)
    
    x2[idx], y2[idx], u2[idx], v2[idx] = x, y, u, v
    idx += 1

Optical flow: 100%|██████████| 1175/1175 [02:13<00:00,  8.78it/s]


In [189]:
plt.close('all')
_ = plot_flow(u2, v2, grid_spacing=32, fn=None)

Traceback (most recent call last):
  File "C:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "C:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\animation.py", line 924, in _stop
    self.event_source.remove_callback(self._step)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'remove_callback'


<IPython.core.display.Javascript object>

Course-to-fine optical flow:

In [190]:
x3, y3, u3, v3 = ctf_optical(u, num_passes=5, scale_factor=1/2)

Optical flow: 100%|██████████| 1/1 [00:37<00:00, 37.31s/it]


In [193]:
plt.close('all')
_ = plot_flow(u3, v3, grid_spacing=32, fn=None)

Traceback (most recent call last):
  File "C:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "C:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\animation.py", line 924, in _stop
    self.event_source.remove_callback(self._step)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'remove_callback'


<IPython.core.display.Javascript object>

# Hybrid method (matching + optical flow)

In [97]:
x4, y4, u4, v4 = match_refine(t, u)

SAD: 100%|██████████| 1175/1175 [02:45<00:00,  7.09it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.38 GiB. GPU 

In [142]:
# plt.close('all')
_ = plot_flow(u4, v4, grid_spacing=32)  # , fn="../experiments/hybrid.gif")

<IPython.core.display.Javascript object>

# Results

In [201]:
testo = pd.read_csv("../Test Data/testo.csv", sep=';')
testo = testo.dropna(how='all').to_numpy()[:, :2]
times = [datetime.strptime(time, '%H:%M:%S') for time in testo[:, 0].flatten()]
velocities = [float(u.replace(',', '.')) for u in testo[:, 1].flatten()]

print(velocities)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.55, 1.01, 1.37, 1.52, 1.52, 1.5, 1.49, 1.52, 1.46, 1.48, 1.44, 1.45, 1.45, 1.47, 1.49, 1.45, 1.48, 1.5, 1.5, 1.48, 1.5, 1.57, 1.53, 1.58, 1.59, 1.55, 1.55, 1.52, 1.6, 1.57, 1.5, 1.57, 1.57, 1.55, 1.55, 1.59, 1.58, 1.5, 1.58, 1.55, 1.57, 1.6, 1.52, 1.57, 1.5, 1.51, 1.51, 1.6, 1.51, 1.52, 1.58, 1.52, 1.62, 1.72, 1.69, 1.72, 1.72, 1.62, 1.61, 1.55, 1.6, 1.6, 1.63, 1.65, 1.65, 1.61, 1.65, 1.62, 1.61, 1.59, 1.68, 1.65, 1.59, 1.7, 1.71, 1.78, 1.82, 1.86, 1.87, 1.87, 1.8, 1.81, 1.81, 1.8, 1.81, 1.82, 1.74, 1.78, 1.73, 1.78, 1.74, 1.74, 1.71, 1.72, 1.9, 1.89, 1.88, 2.0, 1.93, 1.96, 1.93, 1.89, 1.84, 1.86, 1.88, 1.86, 1.91, 1.93, 1.93, 1.93, 1.95, 1.92, 1.92, 1.99, 2.1, 2.1, 2.08, 2.09, 2.09, 2.05, 2.05, 2.02, 2.01, 2.04, 2.04, 1.97, 2.01, 2.06, 2.06, 2.03, 2.11, 2.09, 2.06, 2.06, 2.06, 1.9, 1.42, 0.99, 0.77, 0.77, 0.54, 0.4

In [213]:
def get_datetimes(folder):
    time_objects = []
    
    for file in os.listdir(folder):
        name_formatted = (file[:-4].split("_"))
        name_formatted[-1] = f"{name_formatted[-1]}000"
        name_formatted = [name_formatted[2], name_formatted[1], name_formatted[0], *name_formatted[3:]]
        time = ':'.join(name_formatted[3:6])
        time_objects.append(datetime.strptime(time, '%H:%M:%S'))
    return time_objects

ds = get_datetimes(folder)
print(ds)

[datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 43), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 44), datetime.datetime(1900, 1, 1, 15, 2, 45), datetime.datetime(1900, 1, 1, 15, 2, 45), datetime.datetime(1900, 1, 1, 15, 2, 45), datetime.datetime(1900, 1, 1, 15,

In [224]:
scale = 114.75/30  # px / mm
fps = 170

factor = fps/(scale * 1000)

In [228]:
plt.close('all')

# ref = [1.01, 1.37, 1.52, 1.52, 1.5, 1.49, 1.52, 1.46, 1.48, 1.44, 1.45, 1.45, 1.47, 1.49, 1.45, 1.48, 1.5, 1.5, 1.48]
# idxs = [10497 + i * 240 for i in range(19)]

# img_idx = torch.tensor(t.dataset.idx[::2])
pair_averages = torch.mean(-u1[::2, :, :] * factor, dim=(1, 2)).cpu()

# non_zero_idx = torch.tensor([idx for idx, avg in enumerate(pair_averages.tolist()) if avg != 0.])
# pair_averages = torch.gather(pair_averages, dim=0, index=non_zero_idx)
# img_idx = torch.gather(img_idx, dim=0, index=non_zero_idx)

k = 10  # moving average window size
min_idx, max_idx = 0, len(pair_averages) - 1
avgs = torch.zeros((len(pair_averages),))
stds = torch.zeros((len(pair_averages),))

for idx, avg in enumerate(pair_averages):
    lower, upper = idx - k, idx + k
    lower = min_idx if lower < min_idx else lower
    upper = max_idx if upper > max_idx else upper

    sliced = pair_averages[lower:upper+1]
    # std, moving_avg = torch.std_mean(sliced)
    
    moving_avg = torch.median(sliced)

    avgs[idx] = moving_avg
    # stds[idx] = std / (len(sliced) ** 0.5)
    
# plt.plot(img_idx, avgs, label='Running average', color='blue')
# plt.fill_between(img_idx, avgs - stds, avgs + stds, alpha=0.2, color='blue', label='Standard Error')

# plt.plot(img_idx, pair_averages, linewidth=0.5, linestyle='--', label='measurements')
# plt.plot(idxs, ref, label='Anemometer', color='red')

# plt.plot(ds[::2], pair_averages, label='measurements')
plt.plot(ds[::2], avgs, label='Median filtered')
plt.plot(times, velocities, label='Testo')

plt.xlabel('Timestamp')
plt.ylabel('Velocity (m/s)')

plt.ylim(bottom=0)
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [90]:
vx = -u1[::2] * factor
std, mean = torch.std_mean(vx)

print(f"mean\t{round(mean.item(), 2)}\nstd \t{round(std.item(), 2)}")

mean	2.14
std 	1.02
